# Bayesian Regression
*Curtis Miller*

In the Bayesian approach to regression (also referred to as Bayesian ridge regression, do to an equivalence with ridge regression), the prior distribution of the weights $\beta$ is a Normal distribution. If the error terms $\epsilon_i$ are assumed to be Normally distributed, the posterior distribution of the parameters is also a Normal distribution, with updated parameters. We can make predictions using the **maximum *a posteriori* (MAP)** estimates of the parameters (the values that maximize the posterior distribution's density function).

**Occam's razor** refers to an empirical idea that simple models that explain phenomena are preferred to complex models that explain the same phenomena. This idea appears in Bayesian regression: the prior distribution of the parameters intentionally weights the parameters to zero. This biases the resulting linear model to be "simple", in that features will have negligible weights unless the features have a non-negligible predictive ability. Regression with OLS alone does not have this property; misspecified models will become as complex as necessary to overfit data.

In other words, OLS can be prone to overfitting while Bayesian regression offers an approach to combat overfitting.

## Choosing a Polynomial

Below I load in an artificial dataset consisting of two variables, one of them the target variable.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dat = np.load("mystery_function.npy")
x, y = dat[:, 0], dat[:, 1]
plt.scatter(x, y)
plt.show()

The data in this plot is clearly not linear but could have been generated by some other polynomial relationship. Unfortunately we don't know what polynomial relationship is appropriate and choosing the wrong one can lead to overfitting.

We see this below.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
olsfit1, olsfit2, olsfit3, olsfit4, olsfit5, olsfit6 = (LinearRegression(),)*6

def gen_order_mat(x, order=1):
    """Generates a matrix of x useful for fitting a polynomial of some order"""
    # Similar functionality is supplied by the vander() function in NumPy
    
    if order == 1:
        return x.reshape(-1, 1)
    else:
        return np.array([x**i for i in range(1, order + 1)]).T

# The number designates the order of the fit of the polynomial
olsfit1 = LinearRegression().fit(gen_order_mat(x, order=1), y)
olsfit2 = LinearRegression().fit(gen_order_mat(x, order=2), y)
olsfit3 = LinearRegression().fit(gen_order_mat(x, order=3), y)
olsfit4 = LinearRegression().fit(gen_order_mat(x, order=4), y)
olsfit5 = LinearRegression().fit(gen_order_mat(x, order=5), y)
olsfit10 = LinearRegression().fit(gen_order_mat(x, order=10), y)
olsfit12 = LinearRegression().fit(gen_order_mat(x, order=12), y)

def plotfit(fit, order=1):
    """Plots the function estimated by OLS."""
    
    fx = np.linspace(x.min(), x.max(), num = 100)
    fx_mat = gen_order_mat(fx, order=order)
    yhat = fit.predict(fx_mat)
    plt.scatter(x, y)
    plt.plot(fx, yhat)
    plt.ylim(y.min() - 0.5, y.max() + 0.5)
    plt.show()

plotfit(olsfit1, order=1)

In [ ]:
plotfit(olsfit2, order=2)

In [ ]:
plotfit(olsfit3, order=3)

In [ ]:
plotfit(olsfit4, order=4)

In [ ]:
plotfit(olsfit5, order=5)

In [ ]:
plotfit(olsfit10, order=10)

In [ ]:
plotfit(olsfit12, order=12)

Increasing the order of the polynomial leads to a better fit up until a certain point when new potential features lead to overfitted models. Bayesian ridge regression combats this phenomenon by biasing all parameters to 0, so when fitting a model, parameters get non-negligible contributions to the final fit only when they help in prediction.

## Performing Bayesian Regression

The `BayesianRidge` object allows for performing Bayesian ridge regression.

In [ ]:
from sklearn.linear_model import BayesianRidge
BayesianRidge()

In [ ]:
bayesfit = BayesianRidge(alpha_1 = 1, alpha_2 = 1,
                         lambda_1 = 30, lambda_2 = 50).fit(gen_order_mat(x, order=10), y)

plotfit(bayesfit, order=10)

`alpha_1`, `alpha_2`, `lambda_1`, and `lambda_2` are the hyperparameters of Bayesian ridge regression as supplied by **scikit-learn** (corresponding to parameters of the prior distribution of the parameters). Changing these parameters can lead to better fits. The above function more closely resembles the "best" fit (the cubic curve, which was used to generate the data).